Importando as bibliotecas utilizadas ... 

In [92]:
import numpy as np
import pandas as pd
import nltk
import pickle
import gensim
import spacy

Inicializando o DataFrame da matriz dos conhecimentos, 
utilizando a função 'fillna' para consertar as células mescladas,
lendo a partir da linha 4
e lendo até a coluna 9 (excluso)

In [93]:
matriz_do_conhecimento = pd.read_excel("C:\Coding\WorkSpace\PIBITI\Matriz_do_Conhecimento.xls", skiprows=4).iloc[:, :9].fillna(method="ffill")

Corrigindo valores que foram incorretamente preenchido pelo método fillna("ffill"):

In [94]:
matriz_do_conhecimento = matriz_do_conhecimento.replace({"TÓPICOS": np.NaN, "Nº DE ORDEM DOS TÓPICOS": np.NaN})

Salvando o Dataframe com o pickle

In [95]:
tempFile = open('matrizConhecimentoPickle.txt', 'wb')
pickle.dump(matriz_do_conhecimento, tempFile)
tempFile.close()

Abrindo o dataFrame em formato Binário e Imprimindo o DataFrame para fins de visualização:

In [96]:
tempFile = open('matrizConhecimentoPickle.txt', 'rb', )
matriz_do_conhecimento = pickle.load(tempFile)
tempFile.close()
construcaoEdificacoes_df = matriz_do_conhecimento[ matriz_do_conhecimento['SUB-SETOR'] == 'Construção de Edificações']

Importando as matérias que foram manualmente obtidas e salvas em um arquivo json

In [97]:
materias_df = pd.read_json("C:\Coding\WorkSpace\PIBITI\materias.json")
materias_df

,codigo,nome,ementa,conteudo
0,CIC0004,ALGORITMOS E PROGRAMAÇÃO DE COMPUTADORES,Princípios fundamentais de construção de progr...,
1,IFD0171,FISICA 1,Módulos 1: Unidades e grandezas físicas 2: Vet...,"I-MEDICAO: GRANDEZAS, PADROES E UNIDADES FISIC..."
2,IFD0173,FISICA 1 EXPERIMENTAL,MEDIDAS E ERROS. ANALISE GRAFICA. ATRITO. COLI...,I-CLASSIFICACAO DOS ERROS. CALCULO DE ERRO EXP...
3,MAT0025,CÁLCULO 1,"Funções de uma variável real, limite e continu...",1. Funções: conceito de função exemplo de funç...
4,ENM0133,INTRODUCAO A ENGENHARIA MECATRONICA,"A Universidade de Brasília. Engenharia, Contro...",
5,IQD0125,QUIMICA GERAL TEORICA,Abordagem conceitual dos princípios fundamenta...,1.Estrutura Atômica e a Lei Periódica: O Model...
6,IQD0126,QUIMICA GERAL EXPERIMENTAL,CARACTERIZACAO DA NATUREZA E DO PAPEL DAS INVE...,1) Noções Básicas sobre Segurança no Trabalho ...
7,ENM0131,DESENHO MECANICO ASSISTIDO POR COMPUTADOR 1,Objetivos da Disciplina: A disciplina objetiva...,- Normas de desenho Técnico- Introdução ao CAD...
8,EST0023,PROBABILIDADE E ESTATÍSTICA,"Análise Descritiva, Cálculo de Probabilidades,...",UNIDADE I - Análise Descritiva de Dados - Popu...
9,IFD0175,FISICA 2,Módulos 1: Equilíbrio e elasticidade 2: Gravit...,1-CONTEUDO TEORICO 1-DINAMICA DA ROTACAO 1.1- ...


Pre-processando e removendo certos números e símbolos: " [] () . / - "

In [98]:
from nltk.corpus import stopwords

newtext = {'codigo':[], 'documento':[]}

def preprocess(dado):
    stopWords = stopwords.words("portuguese")
    stopWords += ['i', 'ii', 'iii', 'iv', 'v', 'vi', 'vii', 'viii', 'ix', 'x']

    dado = dado.lower()
    temp = ''
    for character in dado:
        if character not in '.:-^´`()':
            if character not in '1234567890':
                temp += character
    dado = temp
    lista = [j for j in dado.split() if j not in stopWords]
    tempList = []
    for j in lista:
        tempList += gensim.utils.simple_preprocess(j, deacc=True, max_len=40)
    newStr = ' '.join(tempList)
    return newStr

for i, row in materias_df.iterrows():
    codigo = row["codigo"]
    nome = row["nome"]
    ementa = row["ementa"]
    conteudo = row["conteudo"]
    dado = nome + ' ' + ementa + ' ' + conteudo

    newStr = preprocess(dado)
    
    newtext['codigo'].append(codigo)
    newtext['documento'].append(newStr)

materiasPreprocessadas_df = pd.DataFrame(newtext)


Imprimindo o DataFrame para fins de visualização:

In [99]:
materiasPreprocessadas_df

,codigo,documento
0,CIC0004,algoritmos programacao computadores principios...
1,IFD0171,fisica modulos unidades grandezas fisicas veto...
2,IFD0173,fisica experimental medidas erros analise graf...
3,MAT0025,calculo funcoes variavel real limite continuid...
4,ENM0133,introducao engenharia mecatronica universidade...
5,IQD0125,quimica geral teorica abordagem conceitual pri...
6,IQD0126,quimica geral experimental caracterizacao natu...
7,ENM0131,desenho mecanico assistido computador objetivo...
8,EST0023,probabilidade estatistica analise descritiva c...
9,IFD0175,fisica modulos equilibrio elasticidade gravita...


Tokenization and Lemmatization

In [100]:

nlp = spacy.load('pt_core_news_lg', disable=['parser', 'ner'])
lemmatizedData = []
for i, row in materiasPreprocessadas_df.iterrows():
    codigo = row["codigo"]
    text = row["documento"]
    doc = nlp(text)
    lemmatizedData.append([token.lemma_ for token in doc])

construindo o Corpus, vetorizando os documentos com o bag-of-words

In [101]:
from gensim import corpora
id2wordDict = corpora.Dictionary(lemmatizedData)
corpus = [id2wordDict.doc2bow(text) for text in lemmatizedData]

Utilizando tf-idf para enaltecer palavras mais importantes e o LSI

In [102]:
tfidf_model = gensim.models.TfidfModel(corpus, id2word=id2wordDict)
lsi_model = gensim.models.LsiModel(tfidf_model[corpus], id2word=id2wordDict, num_topics=300)

Salvando o corpus já processado  em um arquivo texto do tipo Matrix Market matrix

In [103]:
gensim.corpora.MmCorpus.serialize('tfidf_model_mm', tfidf_model[corpus])
gensim.corpora.MmCorpus.serialize('lsi_model_mm', lsi_model[tfidf_model[corpus]])

Abrindo o corpus salvo no arquivo

In [104]:
tfidf_corpus = gensim.corpora.MmCorpus('tfidf_model_mm')
lsi_corpus = gensim.corpora.MmCorpus('lsi_model_mm')
print(tfidf_corpus)
print(lsi_corpus)

MmCorpus(26 documents, 1154 features, 2011 non-zero entries)
MmCorpus(26 documents, 26 features, 676 non-zero entries)


Buscas de Similaridade Semântica

In [115]:
from gensim.similarities import MatrixSimilarity
from operator import itemgetter

materia_index = MatrixSimilarity(lsi_corpus, num_features = lsi_corpus.num_terms)

def search_similar(search_term):
    search_doc = nlp(search_term)
    lemmatized_doc = [token.lemma_ for token in search_doc]

    query_bow = id2wordDict.doc2bow( lemmatized_doc )
    query_tfidf = tfidf_model[query_bow]
    query_lsi = lsi_model[query_tfidf]

    materia_index.num_best = 5

    listaMaterias = materia_index[query_lsi]

    listaMaterias.sort(key=itemgetter(1), reverse=True)
    resultado = []

    for i, materia in enumerate(listaMaterias):

        resultado.append (
            {
                'Relevancia': round((materia[1] * 100),6),
                'Código da Matéria': materias_df['codigo'][materia[0]],
                'Nome da matéria': materias_df['nome'][materia[0]]
            }

        )
        # ? linha abaixo irrelevante?
        if i == (materia_index.num_best-1):
            break
    
    return pd.DataFrame(resultado, columns=['Relevancia','Código da Matéria','Nome da matéria'])



Utilizando documentos no formato de str, pode-se realizar buscas de similaridade semântica entre as matérias

In [116]:
search_similar('calculo diferencial e integral')

,Relevancia,Código da Matéria,Nome da matéria
0,98.295069,MAT0025,CÁLCULO 1
1,37.565619,MAT0027,CÁLCULO 3
2,32.670581,MAT0028,VARIAVEL COMPLEXA 1
3,15.139326,MAT0026,CÁLCULO 2
4,8.723683,IFD0177,FISICA 2 EXPERIMENTAL


Multi-queries:

In [119]:
construcaoEdificacoes_df.head(15)

for i in range(15):
    texto = construcaoEdificacoes_df.iloc[i, 8]
    texto_processado = preprocess(texto)
    print(f'texto buscado: {texto}')
    print( search_similar(texto_processado), end='\n\n')


texto buscado: Cálculo diferencial e integral 
   Relevancia Código da Matéria        Nome da matéria
0   98.295069           MAT0025              CÁLCULO 1
1   37.565619           MAT0027              CÁLCULO 3
2   32.670581           MAT0028    VARIAVEL COMPLEXA 1
3   15.139326           MAT0026              CÁLCULO 2
4    8.723683           IFD0177  FISICA 2 EXPERIMENTAL

texto buscado: Cálculo numérico 
   Relevancia Código da Matéria        Nome da matéria
0   69.819772           MAT0027              CÁLCULO 3
1   57.942963           MAT0025              CÁLCULO 1
2   33.912727           IFD0177  FISICA 2 EXPERIMENTAL
3   33.201081           IFD0179               FISICA 3
4   31.650785           IFD0173  FISICA 1 EXPERIMENTAL

texto buscado: Álgebra linear 
   Relevancia Código da Matéria              Nome da matéria
0   71.392453           MAT0031  INTRODUCAO A ALGEBRA LINEAR
1   36.505684           CIC0229            CIRCUITOS LÓGICOS
2   30.237508           IFD0173        FISIC